经由pandas写入的csv文件带header；再存入hdfs系统中，之后spark.read.csv无格式问题，可直接有header；直接读会有格式问题；

In [ ]:
import pandas sd pd
df_pd = pd.read_csv("/Users/apple/Desktop/研二/hadoop/PM25city.txt")
df = spark.createDataFrame(df_pd)
df.write.csv(path="hdfs://localhost:8020/user/apple/PM25city_df.csv",header="true")

findsprk package帮助在python环境中设置pyspark的环境变量

In [15]:
import findspark
findspark.init()
import pyspark

初始化spark

In [22]:
spark = pyspark.sql.SparkSession.builder \
    .appName("PM25city") \
    .master("local") \
    .getOrCreate()  

从hdfs系统中读取csv文件，前提是hadoop已经启动；header = true//第一行数据为列名

In [451]:
df = spark.read.csv("hdfs://localhost:8020/user/apple/PM25city_df.csv",header="true")

对数据进行简单的预判

In [421]:
df.show(3)

+-----+------+-----+----+----+---+---+----+-----+---+---+----+----+-----+---+----+----+
| 站号|  经度| 纬度|PM25|PM10|NO2|SO2|O3-1|O3-8h| CO|AQI|等级|year|month|day|hour|city|
+-----+------+-----+----+----+---+---+----+-----+---+---+----+----+-----+---+----+----+
|99000|115.49|38.88|  43|  68| 21| 20| 104|  104|0.6| 60|   2|2018|    8|  1|   0|北京|
|99001|115.51|38.88|  38|  58| 26| 20| 120|  120|0.6| 54|   2|2018|    8|  1|   0|北京|
|99002|115.47|38.91|  50|  72| 22| 17| 113|  113|0.7| 69|   2|2018|    8|  1|   0|北京|
+-----+------+-----+----+----+---+---+----+-----+---+---+----+----+-----+---+----+----+
only showing top 3 rows



In [452]:
df.dtypes

[('站号', 'string'),
 ('经度', 'string'),
 ('纬度', 'string'),
 ('PM25', 'string'),
 ('PM10', 'string'),
 ('NO2', 'string'),
 ('SO2', 'string'),
 ('O3-1', 'string'),
 ('O3-8h', 'string'),
 ('CO', 'string'),
 ('AQI', 'string'),
 ('等级', 'string'),
 ('year', 'string'),
 ('month', 'string'),
 ('day', 'string'),
 ('hour', 'string'),
 ('city', 'string')]

需要修改dataframe里面的数据类型

In [453]:
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,IntegerType
def cast_type_to_int(input_df,col_name):
    return input_df.withColumn(col_name,input_df[col_name].cast(IntegerType()))
a = df.columns
a.remove('city')
print(a)
for n in a:
    df = cast_type_to_int(df, n)
df.dtypes

['站号', '经度', '纬度', 'PM25', 'PM10', 'NO2', 'SO2', 'O3-1', 'O3-8h', 'CO', 'AQI', '等级', 'year', 'month', 'day', 'hour']


[('站号', 'int'),
 ('经度', 'int'),
 ('纬度', 'int'),
 ('PM25', 'int'),
 ('PM10', 'int'),
 ('NO2', 'int'),
 ('SO2', 'int'),
 ('O3-1', 'int'),
 ('O3-8h', 'int'),
 ('CO', 'int'),
 ('AQI', 'int'),
 ('等级', 'int'),
 ('year', 'int'),
 ('month', 'int'),
 ('day', 'int'),
 ('hour', 'int'),
 ('city', 'string')]

In [454]:
df.describe().show()

+-------+------------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------+
|summary|              站号|             经度|              纬度|             PM25|             PM10|               NO2|               SO2|             O3-1|             O3-8h|                CO|              AQI|              等级|              year|             month|               day|              hour|  city|
+-------+------------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------+
|  count|            398355|           398355|            39

发现每一个城市的有行数不一致，可能是因为每一天的测量次数不一致，每个城市的站点也不一致的原因

In [455]:
df.groupBy(df.city).count().collect()

[Row(city='昆明', count=17223),
 Row(city='天津', count=55384),
 Row(city='成都', count=41165),
 Row(city='郑州', count=54302),
 Row(city='厦门', count=22304),
 Row(city='青岛', count=39335),
 Row(city='济南', count=37609),
 Row(city='呼和浩特', count=13728),
 Row(city='上海', count=55441),
 Row(city='海口', count=9398),
 Row(city='乌鲁木齐', count=14838),
 Row(city='北京', count=37628)]

## Problem 1: 拥有最高的和最低的PM25的城市

要确定拥有最高PM25的城市，首先选择有最高值的城市

In [456]:
df.agg({"PM25":"max"}).collect()

[Row(max(PM25)=589)]

In [462]:
df.where(df.PM25==589).collect()

[Row(站号=99011, 经度=116, 纬度=39, PM25=589, PM10=632, NO2=143, SO2=6, O3-1=7, O3-8h=10, CO=4, AQI=500, 等级=6, year=2019, month=1, day=12, hour=21, city='北京'),
 Row(站号=99011, 经度=116, 纬度=39, PM25=589, PM10=632, NO2=143, SO2=6, O3-1=7, O3-8h=10, CO=4, AQI=500, 等级=6, year=2019, month=1, day=12, hour=21, city='天津')]

In [463]:
df.agg({"PM25":"min"}).collect()

[Row(min(PM25)=0)]

In [465]:
df.where(df.PM25==0).groupBy('city').count().sort('count').collect()

[Row(city='海口', count=5),
 Row(city='呼和浩特', count=9),
 Row(city='厦门', count=11),
 Row(city='北京', count=15),
 Row(city='成都', count=28),
 Row(city='天津', count=30),
 Row(city='乌鲁木齐', count=30),
 Row(city='郑州', count=38),
 Row(city='昆明', count=42),
 Row(city='济南', count=43),
 Row(city='青岛', count=63),
 Row(city='上海', count=96)]

计算PM25最高值大于500，属于爆表的城市的天数，然后排序得到结果如下：

In [471]:
gdf = df.groupBy("city","year","month","day")
gdf.agg({"PM25":"max"}).where("max(PM25)>500").groupBy("city").agg({"max(PM25)":"count"}).sort("count(max(PM25))").collect()

[Row(city='郑州', count(max(PM25))=1),
 Row(city='呼和浩特', count(max(PM25))=1),
 Row(city='天津', count(max(PM25))=2),
 Row(city='济南', count(max(PM25))=2),
 Row(city='乌鲁木齐', count(max(PM25))=2),
 Row(city='北京', count(max(PM25))=2)]

### 更换测量标准 选择PM25的最大值小于35，表示该城市的当天空气质量情况良好，
<p>计算每个城市所拥有的天数，得到结果如下：

In [474]:
gdf.agg({"PM25":"max"}).where("max(PM25)<35").groupBy("city").agg({"max(PM25)":"count"}).sort("count(max(PM25))").collect()

[Row(city='上海', count(max(PM25))=1),
 Row(city='济南', count(max(PM25))=2),
 Row(city='厦门', count(max(PM25))=6),
 Row(city='呼和浩特', count(max(PM25))=6),
 Row(city='青岛', count(max(PM25))=7),
 Row(city='昆明', count(max(PM25))=8),
 Row(city='海口', count(max(PM25))=60)]

### 更换测量标准2， 选择一天的PM25的平均值最为该城市的当天PM25的值
<p>并且寻找每个城市的总的PM25的平均值，得到的最终结果排序为

In [179]:
gdf.agg({"PM25":"avg"}).groupBy("city").avg("avg(PM25)").sort("avg(avg(PM25))").collect()

[Row(city='海口', avg(avg(PM25))=15.878023478729292),
 Row(city='昆明', avg(avg(PM25))=24.192705723185576),
 Row(city='厦门', avg(avg(PM25))=26.827465277024345),
 Row(city='呼和浩特', avg(avg(PM25))=39.092328743765975),
 Row(city='上海', avg(avg(PM25))=40.50576370175736),
 Row(city='成都', avg(avg(PM25))=51.023336946955794),
 Row(city='北京', avg(avg(PM25))=55.81112081179114),
 Row(city='青岛', avg(avg(PM25))=56.62600458358438),
 Row(city='天津', avg(avg(PM25))=57.277712989978724),
 Row(city='济南', avg(avg(PM25))=75.25010141092174),
 Row(city='乌鲁木齐', avg(avg(PM25))=76.29316658190142),
 Row(city='郑州', avg(avg(PM25))=88.99536333473276)]

### 更换测量标准3：以每一天的PM25的平均值作为当天的PM25的值，然后计算每个城市平均值小于35的天数
<p>得到结果如下：

In [473]:
gdf.agg({'PM25':"avg"}).where("avg(PM25)<35").groupBy("city").count().sort("count").collect()

[Row(city='郑州', count=16),
 Row(city='济南', count=26),
 Row(city='成都', count=27),
 Row(city='乌鲁木齐', count=31),
 Row(city='天津', count=36),
 Row(city='北京', count=36),
 Row(city='青岛', count=47),
 Row(city='呼和浩特', count=51),
 Row(city='上海', count=57),
 Row(city='厦门', count=72),
 Row(city='昆明', count=80),
 Row(city='海口', count=90)]

### Problem 1 最终结果
可以看见以上的多种测量标准下，最终结果相差不大。
<p>拥有最高的PM25的城市是：郑州、天津、乌鲁木齐、济南、北京、成都</p>
<p>拥有最低的PM25的城市是：海口、昆明、厦门、呼和浩特、上海

## Problem 2 北京、成都、上海的 2019年2月 的城市空气质量分布情况

In [187]:
df.show(3)

+-----+------+-----+----+----+---+---+----+-----+---+---+----+----+-----+---+----+----+
| 站号|  经度| 纬度|PM25|PM10|NO2|SO2|O3-1|O3-8h| CO|AQI|等级|year|month|day|hour|city|
+-----+------+-----+----+----+---+---+----+-----+---+---+----+----+-----+---+----+----+
|99000|115.49|38.88|  43|  68| 21| 20| 104|  104|0.6| 60|   2|2018|    8|  1|   0|北京|
|99001|115.51|38.88|  38|  58| 26| 20| 120|  120|0.6| 54|   2|2018|    8|  1|   0|北京|
|99002|115.47|38.91|  50|  72| 22| 17| 113|  113|0.7| 69|   2|2018|    8|  1|   0|北京|
+-----+------+-----+----+----+---+---+----+-----+---+---+----+----+-----+---+----+----+
only showing top 3 rows



在原数据中有等级这一指标，该指标所有数值为整数值，最大为6；符合老师所设定的六个等级：
<p>Good, Moderate, Unhealthy, Very Unhealthy and Hazardous </p>
<p>

由于spark sql下 GroupedData下的方法并不多，所以spark提供了用户自定义函数，

解决该问题的基本思路是选择北京、上海、成都的2019年2月的数据，
<p>确定每天的等级，然后记数
    <p>每天的等级直接取平均值，将会得到一个浮点数，所以不能直接取平均值，这里自定义函数为取平均值之后，取整数部分

In [477]:
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

def func(x):
    xx = list(x)
    r = int(mean(xx))
    return r
myUDF = F.udf(func,IntegerType())

In [478]:
df2 = df.where(((df.city =="北京") | (df.city=="成都") | (df.city=="上海"))&(df.year==2019)&(df.month==2))
df3 = df2.groupBy("city","day").agg(myUDF(F.collect_list("等级")).alias("rank"))

In [481]:
df3.groupBy("city","rank").agg({"day":"count"}).sort("city").collect()

[Row(city='上海', rank=3, count(day)=3),
 Row(city='上海', rank=1, count(day)=16),
 Row(city='上海', rank=2, count(day)=9),
 Row(city='北京', rank=1, count(day)=11),
 Row(city='北京', rank=3, count(day)=6),
 Row(city='北京', rank=2, count(day)=9),
 Row(city='北京', rank=4, count(day)=2),
 Row(city='成都', rank=1, count(day)=14),
 Row(city='成都', rank=2, count(day)=13),
 Row(city='成都', rank=3, count(day)=1)]

### Problem 2 结果为：
| city | good |  Moderate | Unhealthy | Very | Unhealthy | Hazardous |
| :--: | :--: | :-------: | :-------: | :--: | :-------: | :-------: |
| 北京 | 11 | 9 | 6 | 2 | 0 | 0 |
| 上海 | 16 | 9 | 3 | 0 | 0 | 0 |
| 成都 | 14 | 13 | 1 | 0 | 0 | 0 |


## Problem 3 城市空气质量的可视化，在文件PlotCityMap.ipynb中实现

In [482]:
df.show(3)

+-----+----+----+----+----+---+---+----+-----+---+---+----+----+-----+---+----+----+
| 站号|经度|纬度|PM25|PM10|NO2|SO2|O3-1|O3-8h| CO|AQI|等级|year|month|day|hour|city|
+-----+----+----+----+----+---+---+----+-----+---+---+----+----+-----+---+----+----+
|99000| 115|  38|  43|  68| 21| 20| 104|  104|  0| 60|   2|2018|    8|  1|   0|北京|
|99001| 115|  38|  38|  58| 26| 20| 120|  120|  0| 54|   2|2018|    8|  1|   0|北京|
|99002| 115|  38|  50|  72| 22| 17| 113|  113|  0| 69|   2|2018|    8|  1|   0|北京|
+-----+----+----+----+----+---+---+----+-----+---+---+----+----+-----+---+----+----+
only showing top 3 rows



In [483]:
map0 = df.select(df.经度.alias("lo"),df.纬度.alias("la"),"city","AQI","year","month","day","hour")

In [488]:
map1 = map0.groupBy("city","year","month","day","lo","la").agg({"AQI":"avg"})
map2 = map1.groupBy("city","year","month","day","lo","avg(AQI)").agg({"la":"avg"})
map3 = map2.groupBy("city","year","month","day","avg(la)","avg(AQI)").agg({"lo":"avg"})

In [490]:
map3.withColumnRenamed("avg(AQI)","AQI")
map3.withColumnRenamed("avg(lo)","lo")
map3.withColumnRenamed("avg(la)","la")

map3.describe().show()

+-------+----+------------------+------------------+-----------------+-----------------+-----------------+------------------+
|summary|city|              year|             month|              day|          avg(la)|         avg(AQI)|           avg(lo)|
+-------+----+------------------+------------------+-----------------+-----------------+-----------------+------------------+
|  count|4217|              4217|              4217|             4217|             4217|             4217|              4217|
|   mean|null|2018.6582878823808|3.8541617263457435|15.17714014702395| 33.5515769504387|81.56531827013755|112.29689352620346|
| stddev|null|0.4743398584923505| 3.119432781454499|8.571009844294302|6.172316693250889|58.50447141373459| 8.789633596095099|
|    min|上海|              2018|                 1|                1|             19.0|            11.75|              87.0|
|    max|青岛|              2019|                 8|               31|             44.0|            500.0|             121

In [493]:
map3.repartition(1).write.csv("../resource/city_map.csv",header="true")